In [6]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.merge import add

from keras import regularizers

# Model Definition

## Encoder
The role of this model is to create feature vector from raw images.

Borrow from https://github.com/uzh-rpg/rpg_public_dronet

In [25]:
class ResNet8(keras.Model):
    def __init__(self):
        super(ResNet8, self).__init__(name='')
        
        self.conv2d = Conv2D(32, (5, 5), strides=[2,2], padding='same')
        self.maxpool = MaxPooling2D(pool_size=(3, 3), strides=[2,2])

        # First residual block
        self.r1_bn_1 = keras.layers.normalization.BatchNormalization()
        self.r1_act_1 = Activation('relu')
        self.r1_conv2d_1 = Conv2D(32, (3, 3), strides=[2,2], padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(1e-4))

        self.r1_bn_2 = keras.layers.normalization.BatchNormalization()
        self.r1_act_2 = Activation('relu')
        self.r1_conv2d_2 = Conv2D(32, (3, 3), padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(1e-4))

        self.r1_conv2d_3 = Conv2D(32, (1, 1), strides=[2,2], padding='same')
        #x3 = add([x1, x2])

        # Second residual block
        self.r2_bn_1 = keras.layers.normalization.BatchNormalization()
        self.r2_act_1 = Activation('relu')
        self.r2_conv2d_1 = Conv2D(64, (3, 3), strides=[2,2], padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(1e-4))

        self.r2_bn_2 = keras.layers.normalization.BatchNormalization()
        self.r2_act_2 = Activation('relu')
        self.r2_conv2d_2 = Conv2D(64, (3, 3), padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(1e-4))

        self.r2_conv2d_3 = Conv2D(64, (1, 1), strides=[2,2], padding='same')
        #x5 = add([x3, x4])

        # Third residual block
        self.r3_bn_1 = keras.layers.normalization.BatchNormalization()
        self.r3_act_1 = Activation('relu')
        self.r3_conv2d_1 = Conv2D(128, (3, 3), strides=[2,2], padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(1e-4))

        self.r3_bn_2 = keras.layers.normalization.BatchNormalization()
        self.r3_act_2 = Activation('relu')
        self.r3_conv2d_2 = Conv2D(128, (3, 3), padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(1e-4))

        self.r3_conv2d_3 = Conv2D(128, (1, 1), strides=[2,2], padding='same')
        #x7 = add([x5, x6])

        self.flatten = Flatten()
    
    def build(self, input_shape):
        super(ResNet8, self).build(input_shape)  # Be sure to call this at the end
        
    def call(self, input_tensor, training=False):
        x1 = self.conv2d(input_tensor)
        x1 = self.maxpool(x1)
        
        # First residual block
        x2 = self.r1_bn_1(x1)
        x2 = self.r1_act_1(x2)
        x2 = self.r1_conv2d_1(x2)
        
        x2 = self.r1_bn_2(x2)
        x2 = self.r1_act_2(x2)
        x2 = self.r1_conv2d_2(x2)
        
        x1 = self.r1_conv2d_3(x1)
        
        x3 = add([x1, x2])
        
        # Second residual block
        x4 = self.r2_bn_1(x3)
        x4 = self.r2_act_1(x4)
        x4 = self.r2_conv2d_1(x4)
        
        x4 = self.r2_bn_2(x4)
        x4 = self.r2_act_2(x4)
        x4 = self.r2_conv2d_2(x4)
        
        x3 = self.r2_conv2d_3(x3)
        
        x5 = add([x3, x4])
        
        # Third residual block
        x6 = self.r3_bn_1(x5)
        x6 = self.r3_act_1(x6)
        x6 = self.r3_conv2d_1(x6)
        
        x6 = self.r3_bn_2(x6)
        x6 = self.r3_act_2(x6)
        x6 = self.r3_conv2d_2(x6)
        
        x5 = self.r3_conv2d_3(x5)
        x7 = add([x5, x6])
        
        x = self.flatten(x7)
        
        return x
    
    def compute_output_shape(self, input_shape):
        return (2048,)
        

def pathPlanningModel(img_width, img_height, img_channels, 
                      len_spatial_history, num_classes, planning_horizon=25):
    """
    Define model architecture.
    
    # Arguments
       img_width: Target image widht.
       img_height: Target image height.
       img_channels: Target image channels.
       
    # Returns
       model: A Model instance.
    """
    #------------- Encoder: ResNet8  ---------------#
    # Input
    img_input = Input(shape=(img_height, img_width, img_channels))
    nested_model = ResNet8()
    y = nested_model(img_input)
    
    model = Model(inputs=[img_input], outputs=[y])
    
    print(model.summary())
    
    return model
    

In [26]:
model = pathPlanningModel(128, 128, 1, 3, 400)
# model.load_weights('./model/model_weights.h5', by_name=True)

# # Freeze weights of encoder
# for layer in model.layers:
#     layer.trainable = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 128, 128, 1)       0         
_________________________________________________________________
resnet8_4 (ResNet8)          (2048,)                   309088    
Total params: 309,088
Trainable params: 308,384
Non-trainable params: 704
_________________________________________________________________
None


## Decoder

This model decode the feature vector created by the Encoder to produce the prediction for the `PLANNING_HORIZON` (in form of classes of steering angles) 

In [9]:
def pathPlanningModel(encoder, len_spatial_history, img_width, img_height, img_channels, 
                      num_classes, planning_horizon=25):
    """
    Define the model on top of resnet 8 which produces the steering angles applied to the PLANNING_HORIZON
    
    Input:
        cnn_model (keras model): the feature extraction model
        spatial_history (list of images): 3 frames with 4 meters between 2 adjacent frame
        planning_horizion (int): number of output steering angles
    
    Output:
        keras model instance 
    """
    # Input 
    spatial_history = Input(shape=(len_spatial_history, img_width, img_height, img_channels))
    
    # pass imgs in spatial_history through encoder
    x = [encoder(spatial_history[i, :, :, :]) for i in range(len_spatial_history)]
    
    # concatenate x to make a single tensor
    x = Concatenate()(x)
    
    # dropout
    x = Dropout(0.5)(x)
    
    # 1st Dense layer - 1164 units
    x = Dense(1164, activation='relu')(x)
    
    # 2nd Dense layer - 500 units
    x = Dense(500, activation='relu')(x)
    
    # 3rd Dense layer - num_classes units
    x = Dense(num_classes, activation=None)(x)
    
    # softmax layer
    y = [Activation('softmax')(x) for i in range(planning_horizon)]
    
    # concatenate y to make a single tensor
    y = Concatenate()(y)
    
    # Define steering model
    decoder = Model(inputs=[spatial_history], outputs=[x])
    print(decoder.summary())

    return decoder

In [10]:
decoder = pathPlanningModel(model, 3, 200, 200, 1, 400)

ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: [<tf.Tensor 'model_3_3/flatten_3/Reshape:0' shape=(3, 6272) dtype=float32>, <tf.Tensor 'model_3_4/flatten_3/Reshape:0' shape=(3, 6272) dtype=float32>, <tf.Tensor 'model_3_5/flatten_3/Reshape:0' shape=(3, 6272) dtype=float32>]